<a href="https://colab.research.google.com/github/minasoto54/McCullock-Pitts/blob/main/Redes_multicapa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### PRESENTACIÓN
# REDES MULTICAPA

In [ ]:
"""
Redes Neuronales Multicapa con TensorFlow
===================================================
Ejemplos prácticos divididos en cinco partes:

1️ XOR (problema clásico de aprendizaje no lineal)
2️ Clasificación de la flor de Iris
3️ Clasificación de un candidato a un empleo
4️ Predicción del rendimiento de combustibles
5️ Predicción de popularidad de una canción

Librerías: TensorFlow, NumPy, Scikit-Learn
"""

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.datasets import load_iris, make_classification, make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report, mean_squared_error

# Reproducibilidad
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# === Función auxiliar para crear el modelo MLP # ===
def build_mlp(input_dim, hidden_layers, output_units, output_activation=None):
    model = models.Sequential()
    model.add(layers.Input(shape=(input_dim,)))
    for units in hidden_layers:
        model.add(layers.Dense(units, activation='relu'))
    model.add(layers.Dense(output_units, activation=output_activation))
    return model

# === PARTE 1️ - XOR ===
print("\n   =  =  =  =  =  =")
print("1️  RED NEURONAL MULTICAPA - XOR")
print("  =  =  =  =  =  =")

X = np.array([[0,0],[0,1],[1,0],[1,1]], dtype=float)
y = np.array([0,1,1,0], dtype=float)

model_xor = build_mlp(2, (4,), 1, 'sigmoid')
model_xor.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_xor.fit(X, y, epochs=500, verbose=0)

preds = (model_xor.predict(X) > 0.5).astype(int)
print("Entrenamiento completado ")
print("Predicciones:", preds.reshape(-1))
print("Clases reales:", y.astype(int))

# === PARTE 2️ - CLASIFICACIÓN DE IRIS ===
print("\n   =   =   =   =   =   =")
print("2️  CLASIFICACIÓN DE LA FLOR DE IRIS")
print("   =   =   =   =   =   =")

iris = load_iris()
X = iris.data
y = iris.target.reshape(-1,1)

encoder = OneHotEncoder(sparse_output=False)
y_ohe = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_ohe, test_size=0.2, random_state=SEED)
scaler = StandardScaler().fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)

model_iris = build_mlp(4, (64,32), 3, 'softmax')
model_iris.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model_iris.fit(X_train_s, y_train, epochs=150, batch_size=8, verbose=0, validation_data=(X_test_s, y_test))

acc = model_iris.evaluate(X_test_s, y_test, verbose=0)[1]
print(f"Exactitud en prueba: {acc*100:.2f}%")

preds = np.argmax(model_iris.predict(X_test_s), axis=1)
true = np.argmax(y_test, axis=1)
print("\nReporte de clasificación:")
print(classification_report(true, preds, target_names=iris.target_names))

# === PARTE 3️ - CLASIFICACIÓN DE CANDIDATO A EMPLEO ===
print("\n   =   =   =   =   =   =")
print("3️  CLASIFICACIÓN DE UN CANDIDATO A UN EMPLEO")
print("   =   =   =   =   =   =")

X, y = make_classification(n_samples=500, n_features=4, n_informative=3, n_redundant=0, random_state=SEED)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)
scaler = StandardScaler().fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)

model_job = build_mlp(4, (32,16), 1, 'sigmoid')
model_job.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_job.fit(X_train_s, y_train, epochs=100, verbose=0)

acc = model_job.evaluate(X_test_s, y_test, verbose=0)[1]
print(f"Exactitud en prueba: {acc*100:.2f}%")

preds = (model_job.predict(X_test_s) > 0.5).astype(int)
print("\nReporte de clasificación:")
print(classification_report(y_test, preds, digits=4))

# === PARTE 4️ - PREDICCIÓN DE RENDIMIENTO DE COMBUSTIBLE ===
print("\n   =   =   =   =   =   =")
print("4️  PREDICCIÓN DE RENDIMIENTO DE COMBUSTIBLE")
print("   =   =   =   =   =   =")

X, y = make_regression(n_samples=600, n_features=5, n_informative=4, noise=8.0, random_state=SEED)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)
scaler = StandardScaler().fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)

model_fuel = build_mlp(5, (64,32), 1)
model_fuel.compile(optimizer='adam', loss='mse', metrics=['mse'])
model_fuel.fit(X_train_s, y_train, epochs=150, verbose=0)

preds = model_fuel.predict(X_test_s)
mse = mean_squared_error(y_test, preds)
print(f"Error cuadrático medio (MSE): {mse:.3f}")
print("Ejemplo de predicciones:")
for t,p in list(zip(y_test[:5], preds[:5].reshape(-1))):
    print(f"  Verdadero={t:.2f}, Predicho={p:.2f}")

# === PARTE 5️ - POPULARIDAD DE UNA CANCIÓN ===
print("\n   =   =   =   =   =   =")
print("5️  PREDICCIÓN DE POPULARIDAD DE UNA CANCIÓN")
print("   =   =   =   =   =   =")

n = 800
rng = np.random.RandomState(SEED)
tempo = rng.normal(120, 15, size=n)
dance = rng.uniform(0, 1, size=n)
energy = rng.uniform(0, 1, size=n)
duration = rng.normal(210, 40, size=n)
acoustic = rng.uniform(0, 1, size=n)

X = np.vstack([tempo, dance, energy, duration, acoustic]).T
y = (0.02*(tempo-80) + 30*dance + 25*energy - 0.01*(duration-180) - 10*acoustic + rng.normal(0,5,size=n))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)
scaler = StandardScaler().fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)

model_song = build_mlp(5, (64,32), 1)
model_song.compile(optimizer='adam', loss='mse', metrics=['mse'])
model_song.fit(X_train_s, y_train, epochs=200, verbose=0)

preds = model_song.predict(X_test_s)
mse = mean_squared_error(y_test, preds)
print(f"Error cuadrático medio (MSE): {mse:.3f}")
print("Ejemplo de predicciones:")
for t,p in list(zip(y_test[:5], preds[:5].reshape(-1))):
    print(f"  Verdadero={t:.2f}, Predicho={p:.2f}")

print("\n Todos los modelos fueron entrenados y evaluados correctamente.")



   =  =  =  =  =  =
1️  RED NEURONAL MULTICAPA - XOR
  =  =  =  =  =  =
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Entrenamiento completado 
Predicciones: [0 1 0 0]
Clases reales: [0 1 1 0]

   =   =   =   =   =   =
2️  CLASIFICACIÓN DE LA FLOR DE IRIS
   =   =   =   =   =   =
Exactitud en prueba: 100.00%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step

Reporte de clasificación:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      1.00      1.00         9
   virginica       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30


   =   =   =   =   =   =
3️  CLASIFICACIÓN DE UN CANDIDATO A UN EMPLEO
   =   =   =   =   =   =
Exactitud en prueba: 95.00%
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step

Reporte de clasificación:
              precision    recall  f1-score   support

      

In [ ]:
# REDES NEURONALES MULTICAPA (MLP) CON TENSORFLOW/KERAS
# Autor: Pamela Figueroa y Mina Soto
# Descripción: Este archivo muestra cinco ejemplos de redes multicapa con TensorFlow.
# Cada sección está traducida y explicada en español para comprensión y exposición.

# ===============================================================
# Importar librerías necesarias
# ===============================================================
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris

# ===============================================================
# Parte 1: Problema XOR
# ===============================================================
print("\n=========================")
print(" PARTE 1: PROBLEMA XOR")
print("=========================")

# Datos de entrada (X) y salida (y) del problema XOR
X = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([[0],[1],[1],[0]])

# Definir el modelo
modelo_xor = keras.Sequential([
    keras.layers.Dense(8, activation='relu', input_shape=(2,)),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compilar el modelo
modelo_xor.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
modelo_xor.fit(X, y, epochs=500, verbose=0)

# Evaluar el modelo
perdida, exactitud = modelo_xor.evaluate(X, y, verbose=0)
print(f"Precisión del modelo XOR: {exactitud*100:.2f}%")

# Ejemplo de predicciones
print("Predicciones:")
print(X, "→", modelo_xor.predict(X).round())

# ===============================================================
# Parte 2: Clasificación de la Flor Iris
# ===============================================================
print("\n===========================================")
print(" PARTE 2: CLASIFICACIÓN DE LA FLOR IRIS")
print("===========================================")

# Cargar el dataset Iris
iris = load_iris()
X = iris.data
y = iris.target

# Dividir en entrenamiento y prueba
X_entren, X_prueba, y_entren, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos
escalador = StandardScaler()
X_entren = escalador.fit_transform(X_entren)
X_prueba = escalador.transform(X_prueba)

# Crear el modelo
modelo_iris = keras.Sequential([
    keras.layers.Dense(10, activation='relu', input_shape=(4,)),
    keras.layers.Dense(3, activation='softmax')
])

# Compilar y entrenar
modelo_iris.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
modelo_iris.fit(X_entren, y_entren, epochs=100, verbose=0)

# Evaluar
perdida, exactitud = modelo_iris.evaluate(X_prueba, y_prueba, verbose=0)
print(f"Precisión en la clasificación Iris: {exactitud*100:.2f}%")

# ===============================================================
# Parte 3: Clasificación de un Candidato a Empleo
# ===============================================================
print("\n=====================================================")
print(" PARTE 3: CLASIFICACIÓN DE CANDIDATO A UN EMPLEO")
print("=====================================================")

# Crear un conjunto de datos sintético (experiencia, educación, habilidades)
np.random.seed(0)
X = np.random.rand(200, 3)
y = (X[:,0]*0.5 + X[:,1]*0.3 + X[:,2]*0.2 > 0.5).astype(int)

# Dividir los datos
X_entren, X_prueba, y_entren, y_prueba = train_test_split(X, y, test_size=0.2)

# Definir el modelo
modelo_empleo = keras.Sequential([
    keras.layers.Dense(8, activation='relu', input_shape=(3,)),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compilar y entrenar
modelo_empleo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modelo_empleo.fit(X_entren, y_entren, epochs=150, verbose=0)

# Evaluar
perdida, exactitud = modelo_empleo.evaluate(X_prueba, y_prueba, verbose=0)
print(f"Precisión del clasificador de empleo: {exactitud*100:.2f}%")

# ===============================================================
#  Parte 4: Predicción de Rendimiento de Combustible
# ===============================================================
print("\n==============================================")
print(" PARTE 4: PREDICCIÓN DE RENDIMIENTO DE COMBUSTIBLE")
print("==============================================")

# Datos sintéticos (cilindros, peso, potencia)
np.random.seed(42)
X = np.random.rand(300, 3)
y = X[:,0]*10 + X[:,1]*20 + X[:,2]*15 + np.random.rand(300)*2

X_entren, X_prueba, y_entren, y_prueba = train_test_split(X, y, test_size=0.2)

# Modelo de regresión
modelo_combustible = keras.Sequential([
    keras.layers.Dense(16, activation='relu', input_shape=(3,)),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dense(1)
])

modelo_combustible.compile(optimizer='adam', loss='mse', metrics=['mae'])
modelo_combustible.fit(X_entren, y_entren, epochs=150, verbose=0)

# Evaluar
perdida, error_medio = modelo_combustible.evaluate(X_prueba, y_prueba, verbose=0)
print(f"Error absoluto medio en predicción de combustible: {error_medio:.2f}")

# ===============================================================
# Parte 5: Predicción de Popularidad de una Canción
# ===============================================================
print("\n==============================================")
print(" PARTE 5: PREDICCIÓN DE POPULARIDAD DE UNA CANCIÓN")
print("==============================================")

# Datos sintéticos (energía, baile, duración, acústica)
np.random.seed(123)
X = np.random.rand(400, 4)
y = X[:,0]*40 + X[:,1]*30 + X[:,2]*20 + X[:,3]*10 + np.random.rand(400)*5

X_entren, X_prueba, y_entren, y_prueba = train_test_split(X, y, test_size=0.2)

# Modelo de regresión
modelo_cancion = keras.Sequential([
    keras.layers.Dense(16, activation='relu', input_shape=(4,)),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dense(1)
])

modelo_cancion.compile(optimizer='adam', loss='mse', metrics=['mae'])
modelo_cancion.fit(X_entren, y_entren, epochs=150, verbose=0)

# Evaluar
perdida, error_medio = modelo_cancion.evaluate(X_prueba, y_prueba, verbose=0)
print(f"Error absoluto medio en popularidad: {error_medio:.2f}")

print("\n Fin del programa: todos los modelos entrenados y evaluados correctamente.")



 PARTE 1: PROBLEMA XOR


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Precisión del modelo XOR: 100.00%
Predicciones:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
[[0 0]
 [0 1]
 [1 0]
 [1 1]] → [[0.]
 [1.]
 [1.]
 [0.]]

 PARTE 2: CLASIFICACIÓN DE LA FLOR IRIS


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Precisión en la clasificación Iris: 100.00%

 PARTE 3: CLASIFICACIÓN DE CANDIDATO A UN EMPLEO
Precisión del clasificador de empleo: 87.50%

 PARTE 4: PREDICCIÓN DE RENDIMIENTO DE COMBUSTIBLE
Error absoluto medio en predicción de combustible: 0.57

 PARTE 5: PREDICCIÓN DE POPULARIDAD DE UNA CANCIÓN
Error absoluto medio en popularidad: 4.11

 Fin del programa: todos los modelos entrenados y evaluados correctamente.
